In [1]:
import warnings
warnings.simplefilter('ignore')
import gc
import re
import time
import os
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy.stats import entropy, kurtosis
import xgboost as xgb
import seaborn as sns
from xgboost import plot_importance
import matplotlib.pyplot as plt
import pandas as pd
from math import *
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.datasets.samples_generator import make_blobs
import numpy as np
import lightgbm as lgb
from IPython.display import display

# pd.set_option('display.max_columns', None)
# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)
pd.set_option('float_format', lambda x: '%.6f' % x)

%matplotlib inline

In [2]:
pathf = os.path.join("..", "data", "particles")

trainpd = pd.read_csv(os.path.join(pathf, "train.csv"))
print(trainpd.head(1))
trainshape = trainpd.shape
print(trainshape)
eventpd = pd.read_csv(os.path.join(pathf, "event.csv"))
print(eventpd.head(1))
print(eventpd.shape)
testpd = pd.read_csv(os.path.join(pathf, "test.csv"))
testshape = testpd.shape
print(testpd.head(1))
print(testpd.shape)

            x           y  z          t   terror        q  flag  event_id  \
0 -142.500000 -147.500000  0 767.879000 2.029660 1.050520     0         7   

   hit_id  
0       1  
(9473201, 9)
   event_id  nhit  nhitreal     energymc   thetamc     phimc       xcmc  \
0         7   426        70 48348.900000 63.168600 11.098200 -40.830000   

        ycmc  
0 114.030000  
(13315, 8)
            x           y  z          t   terror        q  event_id  hit_id
0 -142.500000 -127.500000  0 848.061000 1.998400 1.150670         9       1
(4086511, 8)


In [3]:
# (k(q,mc)*(t0+L))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+L) = (t+L)^2
# t0 方程 
# a = k(q,mc)^2
# b = 2*L*k(q,mc)^2 -dis*cos(phi)*sin(thmc)
# c = L^2 * k(q,mc)^2 + dis^2 - dis*cos(phi)*sin(thmc)*L - (t+L)^2 
# t0 = (-b +- (b^2 - 4*a*c)^(1/2))/2*a
data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')

data['fx'] = data['x'] - data['xcmc']
data['fy'] = data['y'] - data['ycmc']
data['phimc'] = data['phimc'] * np.pi / 180.
data['fphi'] = np.arctan2(data['fy'], data['fx']) - data['phimc']
data['fdis'] = np.sqrt(data['fx'] ** 2 + data['fy'] ** 2)
data['thetamc'] = data['thetamc'] * np.pi / 180.

data['fsinthmc'] = np.sin(data['thetamc'])
data['fsinthmc_v'] = 1.0/data['fsinthmc']
data['fcosphi'] = np.cos(data['fphi'])
data['fcosphi_v'] = 1.0/data['fcosphi']

data['fcosthmc'] = np.cos(data['thetamc'])
data['fcosthmc_v'] = 1.0/data['fcosthmc']
data['fsinphi'] = np.sin(data['fphi'])
data['fsinphi_v'] = 1.0/data['fsinphi']

data['ftanphi'] = np.tan(data['fphi'])
data['ftanphi_v'] = 1.0/data['ftanphi']
data['ftanthmc'] = np.tan(data['thetamc'])
data['ftanthmc_v'] = 1.0/data['ftanthmc']


# data['ft2'] = data['t'] ** 2
# data['fdis2'] = data['fdis'] ** 2

data['fttrue'] = data['t'] / data['terror']
data['terror_v'] = 1.0 / data['terror']
data['terror_v2'] =data['terror_v'] ** 2 
data['fttrue_v'] = 1.0 / data['fttrue']
data['fttrue2'] = data['fttrue'] ** 2
data['fttrue2_v'] = 1.0 / data['fttrue2'] 
data['nhitratio'] = data['nhit'] / data['nhitreal']
data['nhitratio_v'] = data['nhitratio']
data['energymc_v'] = 1.0 / data['energymc']
data['fenergymc2'] = data['energymc'] ** 2
data['fenergymc2_v'] = 1.0 / data['fenergymc2'] 
# data['q_v'] = 1.0 / data['q']
data['q2'] = data['q']
# data['q2_v'] = 1.0 / data['q2']

del data['fx']
del data['fy']
del data['x']
del data['y']
del data['z']
del data['xcmc']
del data['ycmc']
del data['fphi']
del data['phimc']
del data['nhitreal']
gc.collect()

11

In [4]:
info_new = pd.DataFrame()
info_new["event_id"] = data.groupby(["event_id"])["event_id"].mean()
info_new["fdis_mean"] = data.groupby(["event_id"])["fdis"].mean()
info_new["fdis_std"] = data.groupby(["event_id"])["fdis"].std()
info_new["fdis_stdmean"] = info_new["fdis_std"] / info_new["fdis_mean"]
info_new["ft_min"] = data.groupby(["event_id"])["t"].min()
info_new["ft_max"] = data.groupby(["event_id"])["t"].max()
info_new["t_mean"] = data.groupby(["event_id"])["t"].mean()
info_new["ft_std"] = data.groupby(["event_id"])["t"].std()
info_new["ft_stdmean"] = info_new["ft_std"] / info_new["t_mean"]
info_new["ft_mean"] = (info_new['t_mean']-info_new['ft_min']) / (info_new['ft_max']-info_new['ft_min'])
info_new.reset_index(drop=True, inplace=True)
data = pd.merge(data, info_new, on='event_id', how='left')

# data['ft_rel'] = (data['t']-data['ft_min']) / (data['ft_max']-data['ft_min'])
data['ft_rel'] = data['t'] / data['ft_std']
# data['ft_rel_std'] = data['ft_rel'] / data['ft_std']
data['ft2_rel'] = data['ft_rel'] ** 2
# data['ft2_rel_std'] = data['ft_rel_std'] ** 2
data['ft_rel_v'] = 1.0 / data['ft_rel']
data['ft2_rel_v'] = 1.0 / data['ft2_rel'] 
# (k(q,mc)*(t0+L))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+L) = (t+L)^2
data = data.sort_values(by=['event_id', 'ft_rel']).reset_index(drop=True)
for i in [4, 6, 8, 10, 12]:
# for i in [7, 8, 11,17, 47]:
    data[f'ft_{i}diff'] = data.groupby('event_id')['ft_rel'].diff(periods=i).fillna(0)
    
data['fdis_rel'] = data['fdis'] / data['fdis_mean']
data['fdis_rel_std'] = data['fdis_rel'] / data['fdis_std']
data['fdis2_rel'] = data['fdis_rel'] ** 2
data['fdis2_rel_std'] = data['fdis_rel_std'] ** 2

data['fdis_t_rel'] = data['fdis_rel'] / data['ft_rel']
data['fdis_tmean_rel'] = data['fdis_rel'] / data['ft_mean']
data['fdis_t_rel'].fillna(0, inplace=True)
# data['fdis-t_rel'] = data['fdis_rel'] * data['ft_rel']
# data['fdis-tmean_rel'] = data['fdis_rel'] * data['ft_mean']
# data['fdis_t_normal'] = data['fdis_rel'] / data['ft_normal']
# data['fdis_t_normal'].fillna(0, inplace=True)
# data['fdis-t_normal'] = data['fdis_rel'] * data['ft_normal']

# data = data.sort_values(by=['event_id', 'fdis_rel']).reset_index(drop=True)
# for i in [5, 6, 7, 8, 10, 11,12]:
#     data[f'fdis_rel_{i}diff'] = data.groupby('event_id')['fdis_rel'].diff(periods=i).fillna(0)
data['fx_normal'] = data['fcosphi'] * data['fdis_rel']
# data['fy_normal'] = data['fsinphi'] * data['fdis_rel']

data['fcossin'] = data['fcosphi'] * data['fsinthmc']
data['fdis_relcossin'] =  data['fdis_rel'] * data['fcossin']
data['fdis2_relcossin'] = data['fdis2_rel'] * data['fcossin']
# data['fdis2_rel_stdcossin'] = data['fdis2_rel_std'] * data['fcossin']
data['ft2_relcossin'] = data['ft2_rel'] * data['fcossin']

data['fsincos'] = data['fsinphi'] * data['fcosthmc']
# data['fdis_relsincos'] = data['fdis_rel'] * data['fsincos']
# data['fdis2_relsincos'] = data['fdis2_rel'] * data['fsincos']
# data['fdis2_rel_stdsincos'] = data['fdis2_rel_std'] * data['fsincos']
# data['ft_relsincos'] =  data['ft_rel'] * data['fsincos']
# data['ft2_relsincos'] = data['ft2_rel'] * data['fsincos']

data['fsinsin'] = data['fsinphi'] * data['fsinthmc']
# data['fdis_relsinsin'] =  data['fdis_rel'] * data['fsinsin']
# data['fdis2_relsinsin'] = data['fdis2_rel'] * data['fsinsin']
# data['fdis2_rel_stdsinsin'] = data['fdis2_rel_std'] * data['fsinsin']
# data['ft_relsinsin'] =  data['ft_rel'] * data['fsinsin']
# data['ft2_relsinsin'] = data['ft2_rel'] * data['fsinsin']

data['fcoscos'] = data['fcosphi'] * data['fcosthmc']

data['ft_relsinsin'] = data['ft_rel'] * data['fsinsin']
data['ft_relcoscos'] = data['ft_rel'] * data['fcoscos']
data['ft_relsincos'] = data['ft_rel'] * data['fsincos']
data['ft_relcossin'] = data['ft_rel'] * data['fcossin']

data['ft_rel_errcoscos'] = data['ft_relcoscos'] * data['terror']
data['fdis2_rel_stdcoscos'] = data['fdis2_rel_std'] * data['fcoscos']
data['ft2_relcoscos'] = data['ft2_rel'] * data['fcoscos']

data['fdis_t_relcossin'] = data['fdis_relcossin'] / data['ft_rel']
data['fdis_t_relcossin'].fillna(0, inplace=True)
data['fdis-t_relcossin'] = data['fdis_relcossin'] * data['ft_rel']

# # 要保留
# del data['ft_7diff']
# del data['ft_8diff']
# del data['ft_11diff']
# del data['ft_17diff']
# del data['fdis_t_rel']
# del data['fttrue2']
# del data['ft_rel_errcoscos']
# del data['fdis_t_relcossin']
# del data['fdis-t_relcossin']
# del data['q2']
# del data['q']
# del data['ft_47diff']
# del data['ft2_rel_v']
# del data['fttrue2_v']
# del data['fttrue_v']
# del data['fsinthmc_v']
# del data['ft_rel']

del data['t']
del data['ft_relcossin']
del data['ft2_rel']
del data['ft_relcoscos']
del data['ft_rel_v']
del data['fcosphi_v']
del data['ft2_relcossin']
del data['ft2_relcoscos']
del data['ftanphi']
del data['ft_relsincos']
del data['ft_relsinsin']
del data['fttrue']
del data['ftanphi_v']
del data['fsinphi_v']

# 要保留但不明显
del data['fdis']
del data['fdis2_rel']
del data['fdis_tmean_rel']
del data['terror_v2']
del data['nhitratio']
del data['fdis2_relcossin']
del data['fdis2_rel_stdcoscos']
del data['ftanthmc']
del data['energymc_v']
del data['nhitratio_v']
del data['fdis_rel_std']
del data['fenergymc2']
del data['fdis2_rel_std']
del data['fdis_stdmean']

# 真删除
del data['fx_normal']
del data['fcossin']
del data['fdis_relcossin']
del data['terror_v']
del data['ft_std']
del data['fdis_rel']
del data['ftanthmc_v']
del data['fenergymc2_v']
del data['fcoscos']
del data['fcosthmc_v']
del data['fdis_mean']
del data['ft_mean']
del data['nhit']
del data['fdis_std']
del data['t_mean']
del data['energymc']
del data['terror']
del data['ft_stdmean']
del data['ft_min']
del data['ft_max']
del data['fcosphi']
del data['thetamc']
del data['fsinthmc']
del data['fcosthmc']
del data['fsincos']
del data['fsinsin']
del data['fsinphi']

gc.collect()

0

In [5]:
print(trainshape[0])
print(data.shape)
testpd = data[data.flag.isna()].reset_index()
# data.loc[data.flag.isna() & (data.t < -900), 'flag'] = 0
# data.loc[data.flag.isna() & ((data.t > 1850) | (data.q < 0)), 'flag'] = 1
trainpd = data[data.flag.notna()].reset_index()
trainpd['flag'] = trainpd['flag'].astype('int')
# trainpd = data[:trainshape[0]].reset_index()
# testpd = data[trainshape[0]:].reset_index()
del data
gc.collect()

9473201
(13559712, 20)


0

In [6]:
print(trainpd.columns)
feature = [x for x in trainpd.columns if x not in ['flag', 'index', 'hit_id', 'event_id']]
# feature = [x for x in trainpd.columns if x not in ['flag', 'index', 'event_id']]
labels = trainpd['flag']

del trainpd['flag']
del testpd['flag']

Index(['index', 'q', 'flag', 'event_id', 'hit_id', 'fsinthmc_v', 'fttrue_v',
       'fttrue2', 'fttrue2_v', 'q2', 'ft_rel', 'ft2_rel_v', 'ft_4diff',
       'ft_6diff', 'ft_8diff', 'ft_10diff', 'ft_12diff', 'fdis_t_rel',
       'ft_rel_errcoscos', 'fdis_t_relcossin', 'fdis-t_relcossin'],
      dtype='object')


In [7]:
if "xgb"=="xgb":
    n_splits = 2
#     X, val_X, y, val_y = train_test_split(
#         train_x,
#         train_y,
#         test_size=0.01,
#         random_state=1,
#         stratify=train_y
#     )

#     # xgb矩阵赋值
#     xgb_val = xgb.DMatrix(val_X, label=val_y)
#     xgb_train = xgb.DMatrix(X, label=y)
#     xgb_test = xgb.DMatrix(test_x)

#     # xgboost模型 #####################

#     params = {
#         'booster': 'gbtree',
#         # 'objective': 'multi:softmax',  # 多分类的问题、
#         # 'objective': 'multi:softprob',   # 多分类概率
#         'objective': 'binary:logistic',
#         'eval_metric': 'logloss',
#         # 'num_class': 9,  # 类别数，与 multisoftmax 并用
#         'gamma': 0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
#         'max_depth': 8,  # 构建树的深度，越大越容易过拟合
#         'alpha': 0,   # L1正则化系数
#         'lambda': 10,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
#         'subsample': 0.7,  # 随机采样训练样本
#         'colsample_bytree': 0.5,  # 生成树时进行的列采样
#         'min_child_weight': 3,
#         # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#         # ，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#         # 这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
#         'silent': 0,  # 设置成1则没有运行信息输出，最好是设置为0.
#         'eta': 0.03,  # 如同学习率
#         'seed': 1000,
#         'nthread': -1,  # cpu 线程数
#         'missing': 1,
#         'scale_pos_weight': (np.sum(y==0)/np.sum(y==1))  # 用来处理正负样本不均衡的问题,通常取：sum(negative cases) / sum(positive cases)
#         # 'eval_metric': 'auc'
#     }
#     plst = list(params.items())
#     num_rounds = 2000  # 迭代次数
#     watchlist = [(xgb_train, 'train'), (xgb_val, 'val')]
#     # 交叉验证
#     result = xgb.cv(plst, xgb_train, num_boost_round=200, nfold=4, early_stopping_rounds=200, verbose_eval=True, folds=StratifiedKFold(n_splits=4).split(X, y))
#     # 训练模型并保存
#     # early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
#     model = xgb.train(plst, xgb_train, num_rounds, watchlist, early_stopping_rounds=200)
#     model.save_model('../data/model/xgb.model')  # 用于存储训练出的模型

#     preds = model.predict(xgb_test)

#     # 导出结果
#     threshold = 0.5
#     for pred in preds:
#         result = 1 if pred > threshold else 0
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=4399)
    fy_submission = np.zeros(testshape[0])
#     y_pp_xgb_stacking = np.zeros(len(labels))
    for train_index, test_index in kf.split(trainpd):
        print(">>>", train_index)
        start_time = time.time()
        clf = xgb.XGBClassifier(tree_method='gpu_hist', max_depth=8, learning_rate=0.05, verbosity=3,
                                min_child_weight=1,colsample_bytree=0.7,subsample=1,
                                eval_metric='auc', n_estimators=5000)
#                                 eval_metric='auc', n_estimators=2000, predictor='cpu_predictor')
        clf.fit(
            trainpd[feature].iloc[train_index], labels[train_index],
            eval_set=[(trainpd[feature].iloc[train_index], labels[train_index]),
                      (trainpd[feature].iloc[test_index], labels[test_index])],
            early_stopping_rounds=50,
            verbose=True,
        )
        fig,ax = plt.subplots(figsize=(15,15))
        for i1 in sorted({icon[0]:icon[1]for icon in zip(feature, clf.feature_importances_)}.items(), key=lambda x: -x[1]):
            print(i1)
        xgb.plot_importance(clf, height=0.5,max_num_features=None,ax=ax,importance_type='gain')
        plt.show()
        used_time = (time.time() - start_time) / 3600
        print(f'used_time: {used_time:.2f} hours')
        y_pred = clf.predict(trainpd[feature].iloc[test_index])
        y_predprob = clf.predict_proba(trainpd[feature].iloc[test_index])[:, 1]
#         y_pp_xgb_stacking[test_index] = y_predprob

        auc = metrics.roc_auc_score(labels[test_index], y_predprob)
        print("AUC Score (Train): %f" % auc)

        fy_submission += clf.predict_proba(testpd[feature])[:, 1] / n_splits

>>> [      3       4       5 ... 9473197 9473198 9473199]
[09:18:59] DEBUG: /workspace/src/tree/updater_gpu_hist.cu:1167: [GPU Hist]: Configure
[09:19:00] DEBUG: /workspace/src/common/device_helpers.cu:38: Running nccl init on: 9.0
[09:19:00] ======== Monitor: DenseCuts ========
[09:19:00] ======== Monitor: HistogramCuts ========
[09:19:00] ======== Monitor: ellpack_page ========
[09:19:00] WARNING: /workspace/src/common/timer.cc:88: Timer for Quantiles did not get stopped properly.


XGBoostError: [09:19:00] /workspace/src/tree/updater_gpu_hist.cu:1058: Exception in gpu_hist: parallel_for failed: out of memory

Stack trace:
  [bt] (0) /home/aa/anaconda2/envs/pygpu36/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x54) [0x7f877eb40614]
  [bt] (1) /home/aa/anaconda2/envs/pygpu36/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(xgboost::tree::GPUHistMakerSpecialised<xgboost::detail::GradientPairInternal<double> >::Update(xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*, xgboost::DMatrix*, std::vector<xgboost::RegTree*, std::allocator<xgboost::RegTree*> > const&)+0x47f) [0x7f877ee23dcf]
  [bt] (2) /home/aa/anaconda2/envs/pygpu36/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(xgboost::gbm::GBTree::BoostNewTrees(xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*, xgboost::DMatrix*, int, std::vector<std::unique_ptr<xgboost::RegTree, std::default_delete<xgboos

In [ ]:
if "lgbt"!="lgbt":
    def run_lgb(df_train, labels, df_test, use_features):
        target = 'flag'
        oof_pred = np.zeros((len(df_train),))
        y_pred = np.zeros((len(df_test),))
        folds = GroupKFold(n_splits=2)  # 6 折比 5 折好一点, 当然有时间有机器可以试下更多的 folds
        for fold, (tr_ind, val_ind) in enumerate(folds.split(df_train, labels, df_train['event_id'])):
            start_time = time.time()
            print(f'Fold {fold + 1}')
            x_train, x_val = df_train[use_features].iloc[tr_ind], df_train[use_features].iloc[val_ind]
            y_train, y_val = labels.iloc[tr_ind], labels.iloc[val_ind]
            train_set = lgb.Dataset(x_train, y_train)
            val_set = lgb.Dataset(x_val, y_val)
#             https://www.cnblogs.com/infaraway/p/7890558.html
            params = {
                'learning_rate': 0.1,
                'metric': 'auc',
                'objective': 'binary',
                'n_estimators': 5000,
                'n_jobs': -1,
                'seed': 1029,
                'device':'gpu',
#                 'is_unbalance': True,
#                 过拟合
                'max_depth': 8,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.75,
                'bagging_freq': 2,
#                 'min_data_in_leaf':1,
#                 'min_sum_hessian_in_leaf':0.001,
#                 'min_gain_to_split': 0.02,
#                 'min_gain_to_split': 0.004,
                'num_leaves': 64,
                'lambda_l1': 0.5,
                'lambda_l2': 0.5, # 越小l2正则程度越高
            }
            model = lgb.train(params,
                              train_set,
                              num_boost_round=5000,
                              early_stopping_rounds=200,
                              valid_sets=[train_set, val_set],
                              verbose_eval=50)
            oof_pred[val_ind] = model.predict(x_val)
            y_pred += model.predict(df_test[use_features]) / folds.n_splits

            print("Features importance...")
            gain = model.feature_importance('gain')
            print(gain)
            feat_imp = pd.DataFrame({'feature': model.feature_name(),
                                     'split': model.feature_importance('split'),
                                     'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
            display(feat_imp)
            used_time = (time.time() - start_time) / 3600
            print(f'used_time: {used_time:.2f} hours')
            del x_train, x_val, y_train, y_val, train_set, val_set
            gc.collect()
        return y_pred, oof_pred
#     lentht = trainpd.shape[0]
#     print(trainpd.shape)
#     fy_submission, oof_pred = run_lgb(trainpd.iloc[0:lentht//100], labels.iloc[0:lentht//100], testpd, feature)
    fy_submission, oof_pred = run_lgb(trainpd, labels, testpd, feature)
    score = roc_auc_score(labels, oof_pred)
    print('auc: ', score)
    print(fy_submission)

In [ ]:
if "stack" != "stack":
    '''模型融合中使用到的各个单模型'''
    clfs = [
        RandomForestClassifier(n_estimators=1000, max_depth=8, n_jobs=-1, criterion='gini'),
        RandomForestClassifier(n_estimators=1000, max_depth=8, n_jobs=-1, criterion='entropy'),
        ExtraTreesClassifier(n_estimators=1000, max_depth=8, n_jobs=-1, criterion='gini'),
        ExtraTreesClassifier(n_estimators=1000, max_depth=8, n_jobs=-1, criterion='entropy'),
        svm.SVC(C=0.1, kernel='linear', decision_function_shape='ovr'),
        xgb.XGBClassifier(tree_method='gpu_hist', max_depth=8, learning_rate=0.1, verbosity=3,
                                eval_metric='auc', n_estimators=1000),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=8, n_estimators=1000)
    ]
    '''切分一部分数据作为测试集'''
    train1pd, train2pd ,label1s, label2s = train_test_split(trainpd[feature], labels, test_size=0.33, random_state=2017)
    dataset_blend_1train = np.zeros((train1pd.shape[0], len(clfs)))
    dataset_blend_2train = np.zeros((train2pd.shape[0], len(clfs)))
    dataset_blend_test = np.zeros((testpd.shape[0], len(clfs)))
    '''5折stacking'''
    n_folds = 2
    skf = list(StratifiedKFold(label1s, n_folds))
    for j, clf in enumerate(clfs):
        '''依次训练各个单模型'''
        print(j, clf)
        start_time = time.time() 
        dataset_blend_2train_j = np.zeros((train2pd.shape[0], len(skf)))
        dataset_blend_test_j = np.zeros((testpd.shape[0], len(skf)))
        for i, (train_index, test_index) in enumerate(skf):
            '''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
            # print("Fold", i)
            X_d1, y_d1, X_d2, y_d2 = train1pd[train_index], label1s[train_index], train2pd[test_index], label1s[test_index]
            clf.fit(X_d1, y_d1)
            dataset_blend_1train[test_index, j] = clf.predict_proba(X_d2)[:, 1]
            dataset_blend_2train_j[:, i] = clf.predict_proba(train2pd)[:, 1]
            dataset_blend_test_j[:, i] = clf.predict_proba(testpd[feature])[:, 1]
        '''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
        dataset_blend_2train[:, j] = dataset_blend_2train_j.mean(1)
        dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
        used_time = (time.time() - start_time) / 3600
        print(f'used_time: {used_time:.2f} hours')
        print("val auc Score: %f" % roc_auc_score(label2s, dataset_blend_2train[:, j]))

In [ ]:
if "stack" != "stack":
# clf = LogisticRegression()
    clf2 = GradientBoostingClassifier(learning_rate=0.02, subsample=0.5, max_depth=6, n_estimators=30)
    clf2.fit(dataset_blend_1train, label1s)
    y_submission = clf2.predict_proba(dataset_blend_2train)[:, 1]
    fy_submission = clf2.predict_proba(dataset_blend_test)[:, 1]

    print("Linear stretch of predictions to [0,1]")
    y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
    print("blend result")
    print("val auc Score: %f" % (roc_auc_score(y_predict, y_submission)))
    fy_submission = (fy_submission - fy_submission.min()) / (fy_submission.max() - fy_submission.min())

In [ ]:
if "blend" != "blend":
    '''模型融合中使用到的各个单模型'''
    clfs = [
        RandomForestClassifier(n_estimators=5000, max_depth=8, n_jobs=-1, criterion='gini'),
        RandomForestClassifier(n_estimators=5000, max_depth=8, n_jobs=-1, criterion='entropy'),
        ExtraTreesClassifier(n_estimators=5000, max_depth=8, n_jobs=-1, criterion='gini'),
        ExtraTreesClassifier(n_estimators=5000, max_depth=8, n_jobs=-1, criterion='entropy'),
#         svm.SVC(C=0.1, kernel='linear', decision_function_shape='ovr'),
        xgb.XGBClassifier(tree_method='gpu_hist', max_depth=8, learning_rate=0.1, verbosity=3,
                                eval_metric='auc', n_estimators=1000),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=8, n_estimators=1000)
    ]
    '''切分训练数据集为d1,d2两部分'''
    X_d1, X_d2, y_d1, y_d2 = train_test_split(trainpd[feature], labels, test_size=0.33, random_state=2017)
#     dataset_d1 = np.zeros((X_d1.shape[0], len(clfs)))
    dataset_d2 = np.zeros((X_d2.shape[0], len(clfs)))
    dataset_test = np.zeros((testpd.shape[0], len(clfs)))
    for j, clf in enumerate(clfs):
        '''依次训练各个单模型'''
        print(j, clf)
        start_time = time.time()
        clf.fit(X_d1, y_d1)
        '''使用第1个部分作为预测，第2部分来训练模型，获得其预测的输出作为第2部分的新特征。'''
        dataset_d2[:, j] = clf.predict_proba(X_d2)[:, 1]
        '''对于测试集，直接用这k个模型的预测值作为新的特征。'''
        dataset_test[:, j] = clf.predict_proba(testpd[feature])[:, 1]
#         dataset_test[:, j] = clf.predict_proba(testpd[feature])[:, 1]
        used_time = (time.time() - start_time) / 3600
        print(f'used_time: {used_time:.2f} hours')
        print("val auc Score: %f" % roc_auc_score(y_d2, dataset_d2[:, j]))
        used_time = (time.time() - start_time) / 3600

In [ ]:
if "blend" != "blend":
    '''模型融合中使用到的各个单模型'''
    '''融合使用的模型'''
    # clf = LogisticRegression()
    clf2 = GradientBoostingClassifier(learning_rate=0.02, subsample=0.5, max_depth=6, n_estimators=30)
    clf2.fit(dataset_d2, y_d2)
#     y_submission = clf2.predict_proba(dataset_d2)[:, 1]
    fy_submission = clf2.predict_proba(dataset_test)[:, 1]

#     print("Linear stretch of predictions to [0,1]")
#     y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
#     print("blend result")
#     print("val auc Score: %f" % (roc_auc_score(y_d2, y_submission)))
    fy_submission = (fy_submission - fy_submission.min()) / (fy_submission.max() - fy_submission.min())

In [ ]:
#阈值大概在0.2-0.4之间 本题对召回率较敏感，可适当降低一下阈值
thre = 0.25

#生成提交文件
sub = pd.DataFrame()
sub['hit_id'] = testpd['hit_id']
sub['flag_pred'] = fy_submission
sub['event_id'] = testpd['event_id']
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= thre else 0)
sub.to_csv(os.path.join(pathf, "subsample.csv").format(sub['flag_pred'].mean()), index=False)